## Web scraping

In [ ]:
import requests
from bs4 import BeautifulSoup

# Realizar la solicitud GET a la página web
url = 'https://ebird.org/region/world/regions?yr=BIGDAY_2023a&hsStats_sortBy=num_species&hsStats_o=desc'
response = requests.get(url)

# Crear el objeto BeautifulSoup para analizar el contenido HTML
soup = BeautifulSoup(response.content, 'html.parser')

# Encontrar los elementos que contienen los precios de los productos
bi2 = soup.find_all('span', class_='Heading Heading--h5 u-margin-none')


In [ ]:
top2=list()
for i in bi2:
    top2.append(i.text)
    
print(top2)

In [ ]:
len(top2)

In [ ]:
# Realizar la solicitud GET a la página web
url = 'https://ebird.org/region/world/regions?yr=BIGDAY_2023a&hsStats_sortBy=num_species&hsStats_o=desc'
response = requests.get(url)

# Crear el objeto BeautifulSoup para analizar el contenido HTML
soup = BeautifulSoup(response.content, 'html.parser')

# Encontrar los elementos que contienen los precios de los productos
bi2_e = soup.find_all('td', class_='Table-cell Table-cell--statBar u-prm')



In [ ]:
top2_e=list()
for i in bi2_e:
    top2_e.append(i.text)
    

In [ ]:
#limpiamos los textos incluidos
top2_e
caracteres_a_eliminar = '\t'

tp1 = [elemento.replace(caracteres_a_eliminar, '') for elemento in top2_e]


caracteres_a_eliminar = '\n'

tp2 = [elemento.replace(caracteres_a_eliminar, '') for elemento in tp1] 
print(tp2)

In [ ]:
# Encontrar los elementos que contienen los precios de los productos
bi2_p = soup.find_all('td', class_='Table-cell Table-cell--statBar u-prm u-showForMedium')


In [ ]:
top2_p=list()
for i in bi2_p:
    top2_p.append(i.text)
#limpiamos los textos incluidos
top2_p
caracteres_a_eliminar = '\t'

tp1p = [elemento.replace(caracteres_a_eliminar, '') for elemento in top2_p]

caracteres_a_eliminar = '\n'

tpp = [elemento.replace(caracteres_a_eliminar, '') for elemento in tp1p] 

caracteres_a_eliminar = ','


tp2p = [elemento.replace(caracteres_a_eliminar, '') for elemento in tpp] 
print(tp2p)

In [ ]:
import pandas as pd
data = {
    'Paises': top2,
    'Especies': tp2,
    'Listas': tp2p
}

df = pd.DataFrame(data)


In [ ]:
df.info()

In [ ]:
df

In [ ]:
# importar las librerias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
# visión cualitativa de los datos
from qgrid import show_grid
show_grid(df)

In [ ]:
from funpymodeling.exploratory import status

status(df)

In [ ]:
df_original=df.copy()

In [ ]:
#convertimos
df['Especies']=df['Especies'].astype(float)
df['Listas']=df['Listas'].astype(float)



In [ ]:
status(df)

# Migracion de excel(csv) a python

In [ ]:
#Creacion de un archivo csv
df.to_csv('Paises_bgd.csv', index=False)

In [ ]:
df_1=pd.read_csv('Paises_bgd.csv')

In [ ]:
df_1

In [ ]:
df_1.info()

## app

In [ ]:
#ERROR de prueba es posible no funcionar

import dash
from dash import dcc as dcc
from dash import html as html
import plotly.express as px
import pandas as pd
import plotly.express as px

# Leer el archivo CSV y crear el DataFrame
df_1= pd.read_csv('Paises_bgd.csv')

# Crear la aplicación Dash
app = dash.Dash(__name__)

# Definir el layout del dashboard
app.layout = html.Div([
    html.H1('Dashboard de Datos'),
    dcc.Graph(
        id='grafico-1',
        figure=px.bar(df_1, x='Paises', y='Especies', title='Gráfico 1')
    ),
    dcc.Graph(
        id='grafico-2',
        figure=px.bar(df_1, x='Paises', y='Listas', title='Gráfico 2')
    ),
    # Agrega más gráficos según tus necesidades
])

# Iniciar el servidor de Dash
if __name__ == '__main__':
    app.run_server(debug=True)


# Graficos juntos

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# Crear un DataFrame de ejemplo
data = {
    'Paises': top2,
    'Especies': tp2,
    'Listas': tp2p
}
df = pd.DataFrame(data)

# Crear visualizaciones interactivas utilizando Plotly
fig = go.Figure()
fig.add_trace(go.Bar(name='Población', x=df['Paises'], y=df['Especies']))
fig.add_trace(go.Bar(name='listas', x=df['Paises'], y=df['Listas']))

fig.update_layout(
    title='Datos de la Big Global day',
    xaxis_title='Paises',
    yaxis_title='Valor',
    barmode='stack'
)

# Guardar el dashboard en un archivo HTML
pio.write_html(fig, file='dashboard.html', auto_open=True)


# Graficos separados

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.io import write_html
from sklearn.preprocessing import MinMaxScaler

# Crear un DataFrame de ejemplo
data = {
    'Paises': top2,
    'Especies': tp2,
    'Listas': tp2p
}
df = pd.DataFrame(data)

# Normalizar los valores en la columna 'Especies'
scaler = MinMaxScaler()
df['Especies'] = scaler.fit_transform(df['Especies'].values.reshape(-1, 1))

# Crear subplots en una sola figura
fig = make_subplots(rows=1, cols=len(df.columns)-1, subplot_titles=df.columns[1:])

# Agregar las gráficas a los subplots
for i, column in enumerate(df.columns[1:], start=1):
    fig.add_trace(go.Bar(x=df['Paises'], y=df[column]), row=1, col=i)

    # Configurar título y etiquetas de los ejes
    fig.update_xaxes(title_text='Paises', row=1, col=i)
    fig.update_yaxes(title_text='Valor', row=1, col=i)

    # Ajustar el rango del eje y
    max_value = float(df[column].max())
    fig.update_yaxes(range=[0, max_value * 1.1], row=1, col=i)

# Configurar el título del dashboard
fig.update_layout(title='Datos de la Big Global day - Dashboard')

# Guardar el dashboard en un archivo HTML
write_html(fig, file='dashboard.html', auto_open=True)
